In [2]:
import pandas as pd
import requests
from tqdm import tqdm_notebook

In [3]:
vacancies = []
vacancy_query = "data scientist"

for page_number in tqdm_notebook(range(50)):
    try:
        response = requests.get("https://api.hh.ru/vacancies",
                                params={
                                    "text": vacancy_query,
                                    "page": page_number}
                                )
        json_response = response.json()

        vacancies += json_response["items"]
    except Exception as ex:
        print(ex)

vacancies_dataset = {
    "vacancy_name": [],
    "employer_name": [],
    "city": [],
    "salary": []
}

/var/folders/c9/dmg044v1297crzr51490bbj40000gn/T/ipykernel_18015/3086884276.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page_number in tqdm_notebook(range(50)):


  0%|          | 0/50 [00:00<?, ?it/s]

In [4]:
for vacancy in vacancies:
    vacancies_dataset["vacancy_name"].append(vacancy["name"])
    vacancies_dataset["employer_name"].append(vacancy["employer"]["name"])

    try:
        vacancies_dataset["city"].append(vacancy.get("address", {}).get("city", "no city"))
    except:
        vacancies_dataset["city"].append("no city")

    salary_from = None
    salary_to = None
    salary_currency = None

    try:
        salary_from = vacancy.get('salary', {}).get('from', None)
    except:
        pass

    try:
        salary_to = vacancy.get('salary', {}).get('to', None)
    except:
        pass

    try:
        salary_currency = vacancy.get('salary', {}).get('currency', None)
    except:
        pass

    if salary_currency != 'RUR' or (salary_from is None and salary_to is None):
        vacancies_dataset["salary"].append(None)
    elif salary_from is not None and salary_to is not None:
        vacancies_dataset["salary"].append((salary_from + salary_to) / 2.0)
    elif salary_from is not None:
        vacancies_dataset["salary"].append(salary_from)
    elif salary_to is not None:
        vacancies_dataset["salary"].append(salary_to)

In [5]:
vacancies_dataset = pd.DataFrame(vacancies_dataset)

In [6]:
vacancies_dataset

,vacancy_name,employer_name,city,salary
0,Data Scientist (Middle),Леруа Мерлен,no city,265000.0
1,Специалист Computer Vision (Middle Data Scient...,Медицинские Скрининг Системы,Санкт-Петербург,200000.0
2,Аналитик данных / Data scientist,MNOGOGAMES,Рязань,155000.0
3,Data Scientist / Аналитик больших данных,Бейстрек Рус,Москва,175000.0
4,Data Scientist,Метр Квадратный,no city,250000.0
...,...,...,...,...
761,Business/System Administrator / L2 Engineer,OneForce,no city,60000.0
762,Business/System Administrator / L2 Engineer,OneForce,no city,60000.0
763,Business/System Administrator / L2 Engineer,OneForce,no city,60000.0
764,Marketing Operation Officer,Kikimora Labs,no city,NaN


In [7]:
vacancies_dataset['salary'].mean()

200328.525

In [20]:
moscow_counter = 0

for vacancy in vacancies:
    try:
        city = vacancy.get("area").get("name")

        if city == 'Москва':
            moscow_counter += 1
    except:
        pass

print(moscow_counter / len(vacancies))

0.5522193211488251
